In [1]:
from os import path, makedirs
from shutil import copyfile
import argparse

import pympi
import numpy as np
import pandas as pd

from elan_helpers import (
    extract_timeseries
)
from xml_helpers import template2case_xml, handle_eaf_file
from video_helpers import find_video_file



In [2]:
def add_annotation_new(eaf, tier_name, data_list, start_time_list, end_time_list):
    
    for start_time, end_time, label in zip(
        start_time_list, end_time_list, data_list
        ):
        if start_time == end_time:
            continue
        eaf.add_annotation(tier_name, start_time, end_time, label)
    return None

In [3]:
output_folder = r'/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction'
annotations_home = r'/beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/2014-11-15.csv'

In [4]:
df=pd.read_csv(annotations_home)

In [15]:
start_times=[]
end_time=[]
labels=[]
for index, row in df.iterrows():
    label = row['Predictions']
    #print(row)
    #print(index)
    if label ==0:
        df.drop(index,axis=0,inplace=True)
    else:
        start_frame = int(row['Start Time'])
        start_times.append(start_frame)
        end_frame = int(row['End Time'])
        end_time.append(end_frame)
        labels.append(str(label))

In [16]:
tiers = [
    "Predictions", 
]


In [17]:
eaf_file= r'/beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/11-15.eaf'

In [18]:
dst_eaf = path.join(output_folder,'trial_5.eaf')

In [19]:
eaf = pympi.Elan.Eaf(eaf_file)

In [18]:
tier_names_to_delete = [ "Gesture", "Modesofrepresent","Clause","Transitivity","visibility","audience","action","NA-layer"]

In [11]:
for tier_name in tier_names_to_delete:
    if tier_name in eaf.get_tier_names():
        eaf.remove_tier(tier_name)
        print(f"Tier '{tier_name}' has been deleted.")
    else:
        print(f"Tier '{tier_name}' does not exist.")

Tier 'Gesture' has been deleted.
Tier 'Modesofrepresent' does not exist.
Tier 'Clause' has been deleted.
Tier 'Transitivity' has been deleted.
Tier 'visibility' has been deleted.
Tier 'audience' has been deleted.
Tier 'action' has been deleted.
Tier 'NA-layer' has been deleted.


In [20]:
for tier_name in tiers:
        eaf.add_tier(tier_name)

In [21]:
add_annotation_new(eaf, "Predictions", labels, start_times, end_time)

In [22]:
eaf.to_file(dst_eaf)

# LOOP FOR CSV MERGE

In [1]:
import os
import re
import numpy as np
import pandas as pd

In [2]:
fp1=r'/beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/version_98'
fp2=r'/beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/version_104'
fp3=r'/beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/version_106'
fp4=r'/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/masking/version_162'
op=r'/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/masking/Final_merge'

In [5]:
for elem in os.listdir(fp1):
    print(elem)
    file_1 = os.path.join(fp1,elem)
    file_2 = os.path.join(fp2,elem)
    file_3 =os.path.join(fp3,elem)
    file_4 =os.path.join(fp4,elem)
    df1 =pd.read_csv(file_1)
    df2=pd.read_csv(file_2)
    df3=pd.read_csv(file_3)
    df4=pd.read_csv(file_4)
    df1['Predictions-3VF'] = df1['Predictions'].round(3)
    df1.drop(columns=['Predictions'], inplace=True)
    df2['Predictions'] = df2['Predictions'].round(3)
    df3['Predictions'] = df3['Predictions'].round(3)
    df4['Predictions'] = df4['Predictions'].round(3)
    df1['Predictions-3OF'] = df2['Predictions']
    df1['Predictions-6 channels'] =df3['Predictions']
    df1['Predictions-mask']=df4['Predictions']
    od=os.path.join(op,elem[:-4]+'.csv')
    df1.to_csv(od, index=False)
    

2014-11-15.csv
2014-12-18.csv
2014-12-04.csv
2014-11-20.csv
2014-12-11.csv


In [4]:
df1

,Filename,Ground Truth,Start Time,End Time,Predictions-3VF,Predictions-3OF,Predictions-6 channels
0,000003399_2014-11-11_1930-2276.npz,0.0,0,1300,0.022,0.098,0.174
1,000003400_2014-11-11_1930-2276.npz,0.0,1300,2600,0.037,0.085,0.112
2,000003401_2014-11-11_1930-2276.npz,0.0,2600,3900,0.014,0.040,0.299
3,000003402_2014-11-11_1930-2276.npz,0.0,3900,5200,0.257,0.420,0.204
4,000003403_2014-11-11_1930-2276.npz,0.0,5200,6500,0.052,0.146,0.807
...,...,...,...,...,...,...,...
261,000003660_2014-11-11_1930-2276.npz,0.0,339300,340600,0.038,0.366,0.340
262,000003661_2014-11-11_1930-2276.npz,0.0,340600,341900,0.101,0.221,0.086
263,000003662_2014-11-11_1930-2276.npz,1.0,341900,343200,0.862,0.721,0.477
264,000003663_2014-11-11_1930-2276.npz,0.0,343200,344500,0.443,0.661,0.225


In [5]:
df2

,Filename,Ground Truth,Predictions,Start Time,End Time
0,000003399_2014-11-11_1930-2276.npz,0.0,0.098,0,1300
1,000003400_2014-11-11_1930-2276.npz,0.0,0.085,1300,2600
2,000003401_2014-11-11_1930-2276.npz,0.0,0.040,2600,3900
3,000003402_2014-11-11_1930-2276.npz,0.0,0.420,3900,5200
4,000003403_2014-11-11_1930-2276.npz,0.0,0.146,5200,6500
...,...,...,...,...,...
261,000003660_2014-11-11_1930-2276.npz,0.0,0.366,339300,340600
262,000003661_2014-11-11_1930-2276.npz,0.0,0.221,340600,341900
263,000003662_2014-11-11_1930-2276.npz,1.0,0.721,341900,343200
264,000003663_2014-11-11_1930-2276.npz,0.0,0.661,343200,344500


In [6]:
df3

,Filename,Ground Truth,Predictions,Start Time,End Time
0,000003399_2014-11-11_1930-2276.npz,0.0,0.174,0,1300
1,000003400_2014-11-11_1930-2276.npz,0.0,0.112,1300,2600
2,000003401_2014-11-11_1930-2276.npz,0.0,0.299,2600,3900
3,000003402_2014-11-11_1930-2276.npz,0.0,0.204,3900,5200
4,000003403_2014-11-11_1930-2276.npz,0.0,0.807,5200,6500
...,...,...,...,...,...
261,000003660_2014-11-11_1930-2276.npz,0.0,0.340,339300,340600
262,000003661_2014-11-11_1930-2276.npz,0.0,0.086,340600,341900
263,000003662_2014-11-11_1930-2276.npz,1.0,0.477,341900,343200
264,000003663_2014-11-11_1930-2276.npz,0.0,0.225,343200,344500


# LOOP FOR EAF

In [6]:
from os import path, makedirs
from shutil import copyfile
import argparse
import os

import pympi
import numpy as np
import pandas as pd

from elan_helpers import (
    extract_timeseries
)
from xml_helpers import template2case_xml, handle_eaf_file
from video_helpers import find_video_file


In [7]:
def add_annotation_new(eaf, tier_name, data_list, start_time_list, end_time_list):
    
    for start_time, end_time, label in zip(
        start_time_list, end_time_list, data_list
        ):
        if start_time == end_time:
            continue
        eaf.add_annotation(tier_name, start_time, end_time, label)
    return None

In [8]:
output_folder = r'/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/masking/Output_ELAN'

In [9]:
annotations_home = r'/beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/version_224'

In [10]:
eaf_home=r'/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Final_merge'

In [11]:
version_type='Final_merge'

In [12]:
annotations_dir=os.path.join(annotations_home,version_type)
print(annotations_dir)

/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/masking/Final_merge


In [16]:
for elem in os.listdir(annotations_dir):
    print(elem)
    annot_file=os.path.join(annotations_dir,elem)
    print(annot_file)
    ele=elem.split('.')[0]
    #ele = '_'.join(ele)
    #ele = ele.split('.')[0]
    print(ele)
    eaf_file = os.path.join(eaf_home, ele+ '.eaf')
    df=pd.read_csv(annot_file)
    start_times=[]
    end_time=[]
    labels_1=[]
    labels_2=[]
    labels_3=[]
    labels_4=[]
    labels_5=[]
    labels_6=[]
    labels_7=[]
    labels_8=[]
    start_times_1 =[]
    start_times_2=[]
    start_times_3=[]
    start_times_4=[]
    end_times_1=[]
    end_times_2=[]
    end_times_3=[]
    end_times_4=[]
    
    for index, row in df.iterrows():
        label_1 = row['Predictions-3VF']
        label_2 = row['Predictions-3OF']
        label_3 = row['Predictions-6 channels']
        label_7 = row['Predictions-mask']
        #print(row)
        #print(index)
        start_frame = int(row['Start Time'])
        start_times.append(start_frame)
        end_frame = int(row['End Time'])
        end_time.append(end_frame)
        labels_1.append(str(label_1))
        labels_2.append(str(label_2))
        labels_3.append(str(label_3))
        labels_7.append(str(label_7))
        if label_1 >= 0.5:
            label_4= row['Predictions-3VF']
            start_frame_1=int(row['Start Time'])
            start_times_1.append(start_frame_1)
            end_frame_1 = int(row['End Time'])
            end_times_1.append(end_frame_1)
            labels_4.append(str(label_4))
        if label_2>= 0.5:
            
            label_5= row['Predictions-3OF']
            start_frame_2=int(row['Start Time'])
            start_times_2.append(start_frame_2)
            end_frame_2 = int(row['End Time'])
            end_times_2.append(end_frame_2)
            labels_5.append(str(label_5))
            
        if label_3>=0.5:
            
            label_6= row['Predictions-6 channels']
            start_frame_3=int(row['Start Time'])
            start_times_3.append(start_frame_3)
            end_frame_3 = int(row['End Time'])
            end_times_3.append(end_frame_3)
            labels_6.append(str(label_6))
            
        if label_7>=0.5:
            
            label_8 = row['Predictions-mask']
            start_frame_4=int(row['Start Time'])
            start_times_4.append(start_frame_4)
            end_frame_4 = int(row['End Time'])
            end_times_4.append(end_frame_4)
            labels_8.append(str(label_8))
            
    tiers = [
    "Predictions-3VC","Predictions-3OF","Predictions-6_channels","Predictions-3VC-2","Predictions-3OF-2" ,"Predictions-6_channels-2","Predictions-mask","Predictions-mask-2"
    ]
    dst_eaf = path.join(output_folder,version_type,elem[:-4]+'.eaf')
    os.makedirs(os.path.dirname(dst_eaf), exist_ok=True)
    eaf = pympi.Elan.Eaf(eaf_file)
    for tier_name in tiers:
        eaf.add_tier(tier_name)
    add_annotation_new(eaf, "Predictions-3VC", labels_1, start_times, end_time)
    add_annotation_new(eaf, "Predictions-3OF", labels_2, start_times, end_time)
    add_annotation_new(eaf, "Predictions-6_channels", labels_3, start_times, end_time)
    add_annotation_new(eaf, "Predictions-3VC-2", labels_4, start_times_1, end_times_1)
    add_annotation_new(eaf, "Predictions-3OF-2", labels_5, start_times_2, end_times_2)
    add_annotation_new(eaf, "Predictions-6_channels-2", labels_6, start_times_3, end_times_3)
    add_annotation_new(eaf, "Predictions-mask", labels_7, start_times, end_time)
    add_annotation_new(eaf, "Predictions-mask-2", labels_8, start_times_4, end_times_4)
    
    eaf.to_file(dst_eaf)
    

2014-11-15.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/masking/Final_merge/2014-11-15.csv
2014-11-15
2014-12-18.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/masking/Final_merge/2014-12-18.csv
2014-12-18
2014-12-04.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/masking/Final_merge/2014-12-04.csv
2014-12-04
2014-11-20.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/masking/Final_merge/2014-11-20.csv
2014-11-20
2014-12-11.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/masking/Final_merge/2014-12-11.csv
2014-12-11


# Csv header addition

In [40]:
import csv

In [41]:
csv_file_path = r'/beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/Training_dataset/Version106_FN_FP/version106'

In [42]:
for elem in os.listdir(csv_file_path):
    csv_path= os.path.join(csv_file_path,elem)
    header_row = ['Filename', 'Ground Truth', 'Predictions', 'Start Time', 'End Time']
    data = []
    with open(csv_path, 'r', newline='') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            data.append(row)
    data.insert(0, header_row)
    with open(csv_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(data)
    print("Header row added to", csv_file_path)
    

    

Header row added to /beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/Training_dataset/Version106_FN_FP/version106
Header row added to /beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/Training_dataset/Version106_FN_FP/version106
Header row added to /beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/Training_dataset/Version106_FN_FP/version106
Header row added to /beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/Training_dataset/Version106_FN_FP/version106
Header row added to /beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/Training_dataset/Version106_FN_FP/version106
Header row added to /beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/Training_dataset/Version106_FN_FP/version106
Header row added to /beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/Training_dataset/Version106_FN_FP/version106
Header row added to /beegfs/.global0/ws/sapo684c-sac_space/Testing_from_models/Training_dataset/Version106_FN_FP/version106
Header r

# LOOP FOR EAF ON PREVIOUS UPDATED ELAN

In [25]:
from os import path, makedirs
from shutil import copyfile
import argparse
import os

import pympi
import numpy as np
import pandas as pd

from elan_helpers import (
    extract_timeseries
)
from xml_helpers import template2case_xml, handle_eaf_file
from video_helpers import find_video_file


In [29]:
def add_annotation_new(eaf, tier_name, data_list, start_time_list, end_time_list):
    
    for start_time, end_time, label in zip(
        start_time_list, end_time_list, data_list
        ):
        if start_time == end_time:
            continue
        eaf.add_annotation(tier_name, start_time, end_time, label)
    return None

In [30]:
output_folder = r'/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset/Final_train'

In [31]:
annotations_home = r'/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset'

In [32]:
eaf_home=r'/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset/Final_merge'

In [33]:
version_type='version_162'

In [34]:
annotations_dir=os.path.join(annotations_home,version_type)
print(annotations_dir)

/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset/version_162


In [35]:
for elem in os.listdir(annotations_dir):
    print(elem)
    annot_file=os.path.join(annotations_dir,elem)
    print(annot_file)
    ele=elem.split('.')[0]
    #ele = '_'.join(ele)
    #ele = ele.split('.')[0]
    print(ele)
    eaf_file = os.path.join(eaf_home, ele+ '.eaf')
    df=pd.read_csv(annot_file)
    start_times=[]
    end_time=[]
    labels_1=[]
    labels_4=[]
    
    start_times_1 =[]
   
    end_times_1=[]
  
    
    for index, row in df.iterrows():
        label_1 = row['Predictions']
        #label_2 = row['Predictions-3OF']
        #label_3 = row['Predictions-6 channels']
        #label_7 = row['Predictions-mask']
        #print(row)
        #print(index)
        start_frame = int(row['Start Time'])
        start_times.append(start_frame)
        end_frame = int(row['End Time'])
        end_time.append(end_frame)
        labels_1.append(str(label_1))
        #labels_2.append(str(label_2))
        #labels_3.append(str(label_3))
        #labels_7.append(str(label_7))
        if label_1 >= 0.5:
            label_4= row['Predictions']
            start_frame_1=int(row['Start Time'])
            start_times_1.append(start_frame_1)
            end_frame_1 = int(row['End Time'])
            end_times_1.append(end_frame_1)
            labels_4.append(str(label_4))
        '''if label_2>= 0.5:
            
            label_5= row['Predictions-3OF']
            start_frame_2=int(row['Start Time'])
            start_times_2.append(start_frame_2)
            end_frame_2 = int(row['End Time'])
            end_times_2.append(end_frame_2)
            labels_5.append(str(label_5))
            
        if label_3>=0.5:
            
            label_6= row['Predictions-6 channels']
            start_frame_3=int(row['Start Time'])
            start_times_3.append(start_frame_3)
            end_frame_3 = int(row['End Time'])
            end_times_3.append(end_frame_3)
            labels_6.append(str(label_6))
            
        if label_7>=0.5:
            
            label_8 = row['Predictions-mask']
            start_frame_4=int(row['Start Time'])
            start_times_4.append(start_frame_4)
            end_frame_4 = int(row['End Time'])
            end_times_4.append(end_frame_4)
            labels_8.append(str(label_8))'''
            
    tiers = [
    "Predictions_1300_mask","Predictions_1300_mask-2"]
    dst_eaf = path.join(output_folder,version_type,elem[:-4]+'.eaf')
    os.makedirs(os.path.dirname(dst_eaf), exist_ok=True)
    eaf = pympi.Elan.Eaf(eaf_file)
    for tier_name in tiers:
        eaf.add_tier(tier_name)
    add_annotation_new(eaf, "Predictions_1300_mask", labels_1, start_times, end_time)
    #add_annotation_new(eaf, "Predictions_250_3VC-2", labels_2, start_times, end_time)
    #add_annotation_new(eaf, "Predictions-6_channels", labels_3, start_times, end_time)
    add_annotation_new(eaf, "Predictions_1300_mask-2", labels_4, start_times_1, end_times_1)
    #add_annotation_new(eaf, "Predictions-3OF-2", labels_5, start_times_2, end_times_2)
    #add_annotation_new(eaf, "Predictions-6_channels-2", labels_6, start_times_3, end_times_3)
    #add_annotation_new(eaf, "Predictions-mask", labels_7, start_times, end_time)
    #add_annotation_new(eaf, "Predictions-mask-2", labels_8, start_times_4, end_times_4)
    
    eaf.to_file(dst_eaf)
    

2014-11-19_950-1220.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset/version_162/2014-11-19_950-1220.csv
2014-11-19_950-1220
2014-12-06_757-1066.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset/version_162/2014-12-06_757-1066.csv
2014-12-06_757-1066
2014-12-13_1200-1593.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset/version_162/2014-12-13_1200-1593.csv
2014-12-13_1200-1593
2014-11-29_1846-2042.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset/version_162/2014-11-29_1846-2042.csv
2014-11-29_1846-2042
2014-12-02_1405-1740.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset/version_162/2014-12-02_1405-1740.csv
2014-12-02_1405-1740
2014-11-18_950-1240.csv
/beegfs/.global0/ws/sapo684c-sac_space/ELAN_Mapping_prediction/Train_dataset/version_162/2014-11-18_950-1240.csv
2014-11-18_950-1240
2014-11-12_956-1428.csv
/beegfs/.global0/ws/sapo6